In [ ]:
import os
os.environ["GROQ_API_KEY"]="g"

from groq import Groq
import instructor
from instructor import Instructor
from pydantic import BaseModel
import json
from abc import ABC, abstractmethod
import requests
import fitz

from prompts import DEFAULT_PROMPT, SYSTEM_PROMPT

client = Groq(
            api_key=os.environ.get("GROQ_API_KEY"),
    )

logs = open("logs.txt",'w')

In [7]:
class ReACT(BaseModel):
    thought : str
    action : str
    action_input : str

class Structurize():
    def __init__(self,client, model):
        self.patched_client = instructor.from_groq(client)
        self.model = model
        
    def __call__(self, messages):
        reply = self.patched_client.chat.completions.create(
            model = self.model,
            response_model = ReACT,
            messages = [{"role": "user", "content": f"Extract: {messages}"}],
        )
        return reply

In [8]:
class Agent:
    def __init__(self, client, model, tools, max_steps=10):
        self.client = client
        self.max_steps = max_steps
        self.model = model
        self.history = []
        self.tools = tools
        self.structurizer = Structurize(client, model)
        
    def __call__(self, message):
        for i in range(self.max_steps):
            response = self.run(message)
            # if response.action.lower() == "finish":
            #     print(f"THOUGHT : {response.thought}\n\nACTION : {response.action}\n\nACTION INPUT : {response.action_input}",file=logs,flush=True)
            #     print("Finished.",file=logs,flush=True)
            #     break
            action = next((tool for tool in self.tools if response.action==tool.__class__.__name__.lower()),None)
            if action is None:
                print(f"THOUGHT : {response.thought}\n\nACTION : {response.action}\n\nACTION INPUT : {response.action_input}",file=logs,flush=True)
                print(f"Action {response.action} not found in tools.")
                print(f"Action {response.action} not found in tools.",file=logs,flush=True)
                self.history.append(f"THOUGHT : {response.thought}\n\nACTION : {response.action}\n\nACTION INPUT : {response.action_input}")
                break
            observation = action.run(response.action_input)
            self.history.append(f"THOUGHT : {response.thought}\n\nACTION : {response.action}\n\nACTION INPUT : {response}\n\nOBSERVATION : {observation}")
            print(f"THOUGHT : {response.thought}\n\nACTION : {response.action}\n\nACTION INPUT : {response.action_input}\n\nOBSERVATION : {observation}",file=logs,flush=True)
            if response.action.lower() == "finish":
                print("Finished.",file=logs,flush=True)
                break
            
    def get_tool_desc(self):
        return  "\n".join([f"{tool.__class__.__name__.lower()}: {tool.run.__doc__}" for tool in self.tools])
    
    def run(self, query, max_retries=3):
        for i in range(max_retries):
            try:
                prompt = DEFAULT_PROMPT.format(query=query, history=self.history, tool_descriptions=self.get_tool_desc())
                output = self.client.chat.completions.create(
                        messages=[
                            {
                                "role": "system",
                                "content": SYSTEM_PROMPT,
                            },
                            {
                                "role": "user",
                                "content": prompt,
                            },
                            
                        ],
                        model=self.model,
                    ).choices[0].message.content
                response = self.structurizer(output)
                return response
                        
            except Exception as e:
                print(e)
                print(e,file=logs,flush=True)
                print("Retrying...")
                print("Retrying...",file=logs,flush=True)
                continue
        print("Max retries exceeded.")
        print("Max retries exceeded.",file=logs,flush=True)
        return ReACT(
            thought="",
            action="finish",
            action_input=""
            )
        

In [9]:
class Tool(ABC):
    """
    Abstract base class for tools.

    Methods:
    run(input_params):
    Abstract method to run the tool with given input parameters.
    """
    @abstractmethod
    def run(self, input):
        pass

class Finish(Tool):
    """
    A tool to return the final answer or result.

    Methods:
    run(final_answer):
    Returns the final answer provided to it.
    """

    def run(self, final_answer):
        """
        Returns the final answer provided to it.

        Parameters:
        final_answer : str
        The final answer or result that needs to be returned.
        Your final answer should always be either yes, no, before, after or a named entity.
        **DO NOT ANSWER IN SENTENCE**

        Returns:
        final_answer : str
        The same final answer or result.
        """
        # print("Finish")
        try:
            print(final_answer)
            return final_answer
        except Exception as e:
            print(f"An error occurred: {e}",file=logs,flush=True)
            return f"An error occurred: {e}"
    
class AskUser(Tool):
    """
    A tool that prompts the user for assistance in answering a query.

    Methods:
    run(query):
    Prompts the user with a query to obtain user assistance.
    """

    def run(self, query):
        """
        Prompts the user for help and awaits their input. Use this when the retrieved information is
        not useful or whenever you need user assistance for further directions. DO NOT ask the user to analyse
        the information or for the final answer. This tool is onl intended to get directions from the user

        Parameters:
        query : str
        The query string that encapsulates the information or question needing a user response.

        Returns:
        response : str
        The response provided by the user.
        """
        # print("AskUser")
        try:
            return input(query)
        except Exception as e:
            print(f"An error occurred: {e}",file=logs,flush=True)
            return f"An error occurred: {e}"
    
class FetchPaper(Tool):
    """
    A tool that fetches the research paper from the internet.

    Methods:
    run(arxiv_id):
    Returns the path of the PDF.
    """

    def run(self, arxiv_id):
        """
        Fetches the research paper from the internet using the provided arxiv ID.

        Parameters:
        arxiv_id : str
        The arxiv ID of the research paper that needs to be fetched.
        **DO NOT ask the user to analyse the information or for the final answer. This tool is onl intended to get directions from the user**

        Returns:
        path : str
        The path to the PDF of the research paper.
        """
        # print("FetchPaper")
        url = f"https://arxiv.org/pdf/{arxiv_id}.pdf"
        save_path = f"{os.getcwd()}/Memory/{arxiv_id}.pdf"
        print("Downloading PDF from:", url, file=logs,flush=True)
        try:
            response = requests.get(url)
            response.raise_for_status()  # Check for request errors

            with open(save_path, "wb") as file:
                file.write(response.content)
            
            return f"Downloaded successfully to {save_path}"
        except requests.RequestException as e:
            print(f"Error downloading PDF: {e}", file=logs,flush=True)
            return f"Error downloading PDF: {e}"
            
        
class Parse(Tool):
    """
    A tool that parses the PDF file and extracts the text from it.
    Methods:
    run(path):
    Parses the PDF file and extracts the text from it.
    """

    def run(self, path):
        """
        Reads the PDF file and extracts the text from it.
        
        Parameters:
        path : str
        The path to the PDF file that needs to be parsed.
        
        Returns:
        text : str
        The extracted text from the PDF file.
        """
        # print("Parse")
        try:
            doc = fitz.open(path)
            text = ""
            for page_num in range(min(1,len(doc))):
                page = doc.load_page(page_num)  # Load page
                text += page.get_text() + "\n"  # Extract text from page

            doc.close()
            
            # Clean text
            clean_text = "\n".join([line.strip() for line in text.splitlines() if line.strip()])
            return clean_text
        except Exception as e:
            print(f"An error occurred: {e}",file=logs,flush=True)
            return f"An error occurred: {e}"
            
class AskExpert(Tool):
    """
    This tool can by used to analyse and process text and also summnerize text.
    The tool asks a question to an expert and returns the answer.
    
    Methods:
    run(query):
    Asks the question to the expert and returns the answer.
    """
    def run(self, query):
        """
        Asks the question to the expert and returns the answer.
        Parameters:
        query : str
        The question that needs to be asked to the expert.
        Returns:
        answer : str
        The answer provided by the expert.
        """
        # print("AskQuestion")
        try:
            client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
            chat_completion = client.chat.completions.create(
                                messages=[
                                    {
                                        "role": "user",
                                        "content": query,
                                    }
                                ],
                                model="llama3-70b-8192",
                            )
            return chat_completion.choices[0].message.content
        except Exception as e:
            print(f"An error occurred: {e}",file=logs,flush=True)
            return f"An error occurred: {e}"



In [10]:
agent = Agent(
    client=client,
    model="llama3-70b-8192",
    tools=[
        Finish(),
        AskUser(),
        FetchPaper(),
        Parse(),
        AskExpert()
    ],
    max_steps=10,
)
agent("Summerize the paper with arxiv id 2411.05886")
while True:
    query = input()
    if query.lower() == "exit":
        break
    agent(query)

Extracted abstract: With the rise of marine exploration, underwater imaging has gained significant attention as a research topic. Under-water video enhancement has become crucial for real-time computer vision tasks in marine exploration. However, most existing methods focus on enhancing individual frames and neglect video temporal dynamics, leading to visually poor enhancements. Furthermore, the lack of ground-truth references limits the use of abundant available underwater video data in many applications.
Error code: 413 - {'error': {'message': 'Request too large for model `llama3-70b-8192` in organization `org_01jf5mp65dew4tbw71ektwjwkq` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 6142, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Retrying...
Error code: 413 - {'error': {'message': 'Request too large for model `llama3-70b-8192` in 